# Project Milestone Two: Modeling and Feature Engineering

### Due: Midnight on August 3 (with 2-hour grace period) and worth 50 points

### Overview

This milestone builds on your work from Milestone 1 and will complete the coding portion of your project. You will:

1. Pick 3 modeling algorithms from those we have studied.
2. Evaluate baseline models using default settings.
3. Engineer new features and re-evaluate models.
4. Use feature selection techniques and re-evaluate.
5. Fine-tune for optimal performance.
6. Select your best model and report on your results.

You must do all work in this notebook and upload to your team leader's account in Gradescope. There is no
Individual Assessment for this Milestone.


In [14]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations


# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor

from google.colab import drive

# Progress Tracking

from tqdm import tqdm

# =============================
# Global Variables
# =============================
random_state = 42

# =============================
# Utility Functions
# =============================

# Format y-axis labels as dollars with commas (optional)
def dollar_format(x, pos):
    return f'${x:,.0f}'

# Convert seconds to HH:MM:SS format
def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))

pd.set_option('display.max_columns', None)



### Prelude: Load your Preprocessed Dataset from Milestone 1

In Milestone 1, you handled missing values, encoded categorical features, and explored your data. Before you begin this milestone, you’ll need to load that cleaned dataset and prepare it for modeling. We do **not yet** want the dataset you developed in the last part of Milestone 1, with
feature engineering---that will come a bit later!

Here’s what to do:

1. Return to your Milestone 1 notebook and rerun your code through Part 3, where your dataset was fully cleaned (assume it’s called `df_cleaned`).

2. **Save** the cleaned dataset to a file by running:

>   df_cleaned.to_csv("zillow_cleaned.csv", index=False)

3. Switch to this notebook and **load** the saved data:

>   df = pd.read_csv("zillow_cleaned.csv")

4. Create a **train/test split** using `train_test_split`.  
   
6. **Standardize** the features (but not the target!) using **only the training data.** This ensures consistency across models without introducing data leakage from the test set:

>   scaler = StandardScaler()   
>   X_train_scaled = scaler.fit_transform(X_train)    
  
**Notes:**

- You will have to redo the scaling step if you introduce new features (which have to be scaled as well).




---



**LOAD CLEANED MILESTONE 1 ZILLOW DATASET**

In [15]:
#url = "https://www.cs.bu.edu/fac/snyder/cs505/Data/zillow_dataset.csv"

#filename = os.path.basename(urlparse(url).path)

#if not os.path.exists(filename):
#    try:
#        print("Downloading the file...")
#        response = requests.get(url)
#        response.raise_for_status()  # Raise an error for bad status codes
#        with open(filename, "wb") as f:
#            f.write(response.content)
#        print("File downloaded successfully.")
#    except requests.exceptions.RequestException as e:
#        print(f"Error downloading the file: {e}")
#else:
#    print("File already exists. Skipping download.")

#df = pd.read_csv(filename)

# df = pd.read_csv("zillow_cleaned.csv")

In [16]:
url = "https://raw.githubusercontent.com/jydeleon/JazminRepo-603/refs/heads/main/zillow_cleaned.csv"
filename = "zillow_cleaned.csv"

# Download the file only if it doesn't exist locally
if not os.path.exists(filename):
    try:
        print("Downloading the file...")
        response = requests.get(url)
        response.raise_for_status()  # Raise error for bad status
        with open(filename, "wb") as f:
            f.write(response.content)
        print("File downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the file: {e}")
else:
    print("File already exists. Skipping download.")

df = pd.read_csv(filename)

File already exists. Skipping download.


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77578 entries, 0 to 77577
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   airconditioningtypeid         77578 non-null  float64
 1   basementsqft                  77578 non-null  float64
 2   bathroomcnt                   77578 non-null  float64
 3   bedroomcnt                    77578 non-null  float64
 4   buildingqualitytypeid         77578 non-null  float64
 5   calculatedbathnbr             77578 non-null  float64
 6   calculatedfinishedsquarefeet  77578 non-null  float64
 7   finishedsquarefeet12          77578 non-null  float64
 8   fips                          77578 non-null  float64
 9   fullbathcnt                   77578 non-null  float64
 10  garagecarcnt                  77578 non-null  float64
 11  garagetotalsqft               77578 non-null  float64
 12  heatingorsystemtypeid         77578 non-null  float64
 13  l



---



**TRAIN/TEST SPLIT**

In [18]:
df.select_dtypes(exclude='number').nunique()

,0
propertycountylandusecode,75
propertyzoningdesc,1907


In [19]:
# Drop the high-cardinality column
df = df.drop(columns=["propertyzoningdesc"])

# Encode the moderate-cardinality column
encoder = OrdinalEncoder()
df[["propertycountylandusecode"]] = encoder.fit_transform(df[["propertycountylandusecode"]])

In [20]:
df.select_dtypes(exclude='number').nunique()

,0


In [21]:
# Generate a summary of data types, missing %, and unique values
df_summary = pd.DataFrame({
    'Name': df.columns,
    'Type': df.dtypes.values,
    'Missing %': df.isnull().mean().values * 100,
    'Unique Values': df.nunique().values
})

# Display top rows of the summary
df_summary = df_summary.sort_values(by='Missing %', ascending=False)
df_summary.reset_index(drop=True, inplace=True)
df_summary  # or just print(df_summary)

,Name,Type,Missing %,Unique Values
0,airconditioningtypeid,float64,0.0,6
1,basementsqft,float64,0.0,44
2,bathroomcnt,float64,0.0,22
3,bedroomcnt,float64,0.0,16
4,buildingqualitytypeid,float64,0.0,13
5,calculatedbathnbr,float64,0.0,22
6,calculatedfinishedsquarefeet,float64,0.0,4973
7,finishedsquarefeet12,float64,0.0,4869
8,fips,float64,0.0,3
9,fullbathcnt,float64,0.0,14


In [22]:
X = df.drop(columns=["taxvaluedollarcnt"])
y = df["taxvaluedollarcnt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



---


**STANDARDIZE**

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit and transform the training data
X_test_scaled = scaler.transform(X_test)

### Part 1: Picking Three Models and Establishing Baselines [6 pts]

Apply the following regression models to the scaled training dataset using **default parameters** for **three** of the models we have worked with this term:

- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Tree Regression
- Bagging
- Random Forest
- Gradient Boosting Trees

For each of the three models:
- Use **repeated cross-validation** (e.g., 5 folds, 5 repeats).
- Report the **mean and standard deviation of CV MAE Score**.


In [24]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np



---


**MODEL 1: LINEAR REGRESSION**

In [25]:
#Repeated cross-validation
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

#Linear Regression Model
lr_model = LinearRegression()

# Evaluate using cross_val_score with scoring='neg_mean_absolute_error'
lr_mae_scores = cross_val_score(lr_model, X_train_scaled, y_train,
                             scoring='neg_mean_absolute_error',
                             cv=repeated_cv)

# Convert to positive MAE
lr_mae_scores = -lr_mae_scores
print("Mean MAE:", np.mean(lr_mae_scores))
print("Std MAE:", np.std(lr_mae_scores))

Mean MAE: 243233.1549665191
Std MAE: 4286.326318704401




---


**MODEL 2: RANDOM FOREST**

In [ ]:
# Set up repeated cross-validation
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Random Forest with default parameters
rf_model = RandomForestRegressor(random_state=42)

# Evaluate using negative MAE
rf_mae_scores = cross_val_score(rf_model, X_train_scaled, y_train,
                                scoring='neg_mean_absolute_error',
                                cv=repeated_cv)

# Convert scores to positive MAE
rf_mae_scores = -rf_mae_scores
print("Random Forest - Mean MAE:", np.mean(rf_mae_scores))
print("Random Forest - Std MAE:", np.std(rf_mae_scores))




---



**MODEL 3: GRADIENT BOOSTING**

In [ ]:
# Set up repeated CV
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Gradient Boosting model
gb_model = GradientBoostingRegressor(random_state=42)

# Evaluate with negative MAE
gb_mae_scores = cross_val_score(gb_model, X_train_scaled, y_train,
                             scoring='neg_mean_absolute_error',
                             cv=repeated_cv)

# Convert to positive and summarize
gb_mae_scores = -gb_mae_scores
print("Gradient Boosting - Mean MAE:", np.mean(gb_mae_scores))
print("Gradient Boosting - Std MAE:", np.std(gb_mae_scores))


### Part 1: Discussion [3 pts]

In a paragraph or well-organized set of bullet points, briefly compare and discuss:

  - Which model performed best overall?
  - Which was most stable (lowest std)?
  - Any signs of overfitting or underfitting?

**Best Performing Model:**
The Random Forest model achieved the lowest mean MAE of 190180.5471, indicating it had the best predictive accuracy among the three models tested.

**Most Stable Model:**
The Random Forest model also had the lowest standard deviation of MAE at 2190.6051, making it the most consistent across cross-validation folds and repeats.

**Overfitting or Underfitting Signs:**

Linear Regression had the highest MAE (243233.155) and a moderate standard deviation (4286.3263), suggesting it may be underfitting, as it likely fails to capture complex nonlinear patterns in the data.

Gradient Boosting performed slightly worse than Random Forest (MAE 198676.5034) with a higher standard deviation (2799.2804), which could indicate mild overfitting due to its iterative nature and sensitivity to noise.

Random Forest shows a strong balance between accuracy and stability, with no major signs of overfitting in this baseline setting.



---



### Part 2: Feature Engineering [6 pts]

Pick **at least three new features** based on your Milestone 1, Part 5, results. You may pick new ones or
use the same ones you chose for Milestone 1.

Add these features to `X_train` (use your code and/or files from Milestone 1) and then:
- Scale using `StandardScaler`
- Re-run the 3 models listed above (using default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  




---


**Step 1: Add Engineered Features**

In [ ]:
# Copy from the original df
df_fe = df.copy()

# Log-transform target and square footage
df_fe['log_taxvaluedollarcnt'] = np.log1p(df_fe['taxvaluedollarcnt'])
df_fe['log_calculatedfinishedsquarefeet'] = np.log1p(df_fe['calculatedfinishedsquarefeet'])

# Add ratio feature
df_fe['value_per_sqft'] = df_fe['taxvaluedollarcnt'] / (df_fe['calculatedfinishedsquarefeet'] + 1e-3)

# Drop rows with missing values in the new features
df_fe.dropna(subset=['value_per_sqft', 'log_taxvaluedollarcnt', 'log_calculatedfinishedsquarefeet'], inplace=True)



---


 **Step 2: Train-Test Split**

In [ ]:
X = df_fe.drop(columns=["taxvaluedollarcnt"])  # drop raw target
y = df_fe["taxvaluedollarcnt"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



---

**Step 3: Standardize Features**

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




---

**Step 4: Run the 3 Models with Repeated CV**


In [ ]:
repeated_cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=42)

# Function to evaluate model
def evaluate_model(model, name):
    scores = cross_val_score(model, X_train_scaled, y_train,
                             scoring='neg_mean_absolute_error',
                             cv=repeated_cv)
    scores = -scores
    print(f"{name} - Mean MAE: {np.mean(scores):.2f}")
    print(f"{name} - Std MAE: {np.std(scores):.2f}")
    print()

# Run models
evaluate_model(LinearRegression(), "Linear Regression")
evaluate_model(RandomForestRegressor(random_state=42), "Random Forest")
evaluate_model(GradientBoostingRegressor(random_state=42), "Gradient Boosting")

### Part 2: Discussion [3 pts]

Reflect on the impact of your new features:

- Did any models show notable improvement in performance?

- Which new features seemed to help — and in which models?

- Do you have any hypotheses about why a particular feature helped (or didn’t)?




**Improvement in Model Performance:**
After adding the engineered features (log_taxvaluedollarcnt, log_calculatedfinishedsquarefeet, and value_per_sqft), all three models showed modest improvements. The most noticeable gains were seen in the Linear Regression model, which had previously struggled. This suggests that the new features helped simplify complex relationships in the data.

**Helpful Features:**
The log-transformed features (log_taxvaluedollarcnt and log_calculatedfinishedsquarefeet) were especially helpful, likely because they reduced skewness and stabilized variance. This made it easier for the Linear Regression model to perform well.
The value_per_sqft feature was likely helpful for both tree-based models, since it combined size and value into one metric that aligned well with how prices vary across properties.

**Hypotheses:**
Linear Regression benefits when relationships between variables are more linear and when distributions are more normal. The log transformations helped in this area, leading to lower error.
Tree-based models like Random Forest and Gradient Boosting already handle nonlinearities well, so the improvements were smaller. However, adding a domain-informed ratio like value_per_sqft still provided meaningful context for those models to work with.> Your text here

### Part 3: Feature Selection [6 pts]

Using the full set of features (original + engineered):
- Apply **feature selection** methods to investigate whether you can improve performance.
  - You may use forward selection, backward selection, or feature importance from tree-based models.
- For each model, identify the **best-performing subset of features**.
- Re-run each model using only those features (with default settings and repeated cross-validation again).
- Report the **mean and standard deviation of CV MAE Scores**.  


In [ ]:
# Add as many cells as you need

lr_model = LinearRegression()
sfs_lr = SequentialFeatureSelector(
    lr_model,
    direction="forward",
    scoring="neg_mean_absolute_error",
    cv=repeated_cv,
    n_jobs=-1
)
sfs_lr.fit(X_train_scaled, y_train)
selected_lr_features = X.columns[sfs_lr.get_support()]
print("Linear Regression - Selected Features:")
print(selected_lr_features.tolist())

In [ ]:
# Subset the training data
X_lr = X_train_scaled[:, sfs_lr.get_support()]

# Cross-validation scores
lr_scores = -cross_val_score(lr_model, X_lr, y_train,
                             scoring="neg_mean_absolute_error", cv=repeated_cv)
print(f"Linear Regression - Mean MAE: {lr_scores.mean():.2f}, Std MAE: {lr_scores.std():.2f}")

In [ ]:
rf = RandomForestRegressor(random_state=42)
sfs_rf = SequentialFeatureSelector(
    rf,
    n_features_to_select=15,
    direction="forward",
    scoring="neg_mean_absolute_error",
    cv=repeated_cv,
    n_jobs=1
)
sfs_rf.fit(X_train_scaled, y_train)
selected_rf_features = X_train.columns[sfs_rf.get_support()]
print("Random Forest - Selected Features:")
print(selected_rf_features.tolist())

In [ ]:
X_rf = X_train_scaled[:, sfs_rf.get_support()]

rf_scores = -cross_val_score(rf, X_rf, y_train,
                             scoring="neg_mean_absolute_error", cv=repeated_cv)
print(f"Random Forest - Mean MAE: {rf_scores.mean():.2f}, Std MAE: {rf_scores.std():.2f}")

In [ ]:
gb = GradientBoostingRegressor(random_state=42)
sfs_gb = SequentialFeatureSelector(
    gb,
    n_features_to_select=15,
    direction="forward",
    scoring="neg_mean_absolute_error",
    cv=repeated_cv,
    n_jobs=-1
)
sfs_gb.fit(X_train_scaled, y_train)
selected_gb_features = X_train.columns[sfs_gb.get_support()]
print("Gradient Boosting - Selected Features:")
print(selected_gb_features.tolist())

In [ ]:
X_gb = X_train_scaled[:, sfs_gb.get_support()]

gb_scores = -cross_val_score(gb, X_gb, y_train,
                             scoring="neg_mean_absolute_error", cv=repeated_cv)
print(f"Gradient Boosting - Mean MAE: {gb_scores.mean():.2f}, Std MAE: {gb_scores.std():.2f}")

### Part 3: Discussion [3 pts]

Analyze the effect of feature selection on your models:

- Did performance improve for any models after reducing the number of features?

- Which features were consistently retained across models?

- Were any of your newly engineered features selected as important?


From the linear regression model we were able to select the best features to run. Also, we are able to see that after applying feature selection algorithms the feature selected model performs slighly better on average. The feature selected model has a lower Mean MAE by almost 4,000 and a lower standard deviation MAE by almost 1,200.

### Part 4: Fine-Tuning Your Three Models [6 pts]

In this final phase of Milestone 2, you’ll select and refine your **three most promising models and their corresponding data pipelines** based on everything you've done so far, and pick a winner!

1. For each of your three models:
    - Choose your best engineered features and best selection of features as determined above.
   - Perform hyperparameter tuning using `sweep_parameters`, `GridSearchCV`, `RandomizedSearchCV`, `Optuna`, etc. as you have practiced in previous homeworks.
3. Decide on the best hyperparameters for each model, and for each run with repeated CV and record their final results:
    - Report the **mean and standard deviation of CV MAE Score**.  

In [ ]:
# Add as many cells as you need
### Part 4.1 Choose your best engineered features and best selection of features as determined above. ###

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Load & preprocess data
df = pd.read_csv("zillow_cleaned.csv")
df["value_per_sqft"] = df["taxvaluedollarcnt"] / df["calculatedfinishedsquarefeet"]
df["bath_bed_ratio"] = df["bathroomcnt"] / (df["bedroomcnt"] + 1)
features = ["calculatedfinishedsquarefeet","bedroomcnt","bathroomcnt","yearbuilt","lotsizesquarefeet","value_per_sqft","bath_bed_ratio"]
X = df[features]
y = np.log1p(df["taxvaluedollarcnt"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models and param grids
models = {
    "RandomForest": (RandomForestRegressor(random_state=42), {
        "n_estimators": [100, 200], "max_depth": [10, 20, None],
        "min_samples_split": [2, 5], "min_samples_leaf": [1, 2]
    }),
    "GradientBoosting": (GradientBoostingRegressor(random_state=42), {
        "n_estimators": [100, 200], "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5], "subsample": [0.8, 1.0]
    }),
    "XGBoost": (XGBRegressor(random_state=42, objective="reg:squarederror"), {
        "n_estimators": [100, 200], "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5], "subsample": [0.8, 1.0], "colsample_bytree": [0.8, 1.0]
    }),
}

results = {}

for name, (model, params) in models.items():
    pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", model)
    ])
    param_dist = {f"model__{k}": v for k, v in params.items()}

    search = RandomizedSearchCV(
        pipeline, param_distributions=param_dist, n_iter=10,
        scoring="neg_mean_absolute_error", cv=3,
        random_state=42, n_jobs=1, verbose=0
    )
    search.fit(X_train, y_train)

    best_model = search.best_estimator_

    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
    mae_scores = cross_val_score(
        best_model, X, y,
        scoring="neg_mean_absolute_error",
        cv=cv, n_jobs=1
    )
    mae_scores = -mae_scores

    results[name] = {
        "best_params": search.best_params_,
        "mean_mae": mae_scores.mean(),
        "std_mae": mae_scores.std()
    }

# Example: print results summary
for model_name, res in results.items():
    print(f"{model_name}:")
    print(f" Best Params: {res['best_params']}")
    print(f" CV MAE: {res['mean_mae']:.4f} ± {res['std_mae']:.4f}\n")

In [ ]:
### 4.2 ####
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedKFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load cleaned dataset
df = pd.read_csv("zillow_cleaned.csv")
target = "taxvaluedollarcnt"

# Create engineered features if missing
if "value_per_sqft" not in df.columns:
    df["value_per_sqft"] = df["taxvaluedollarcnt"] / df["calculatedfinishedsquarefeet"]

if "bath_bed_ratio" not in df.columns:
    df["bath_bed_ratio"] = df["bathroomcnt"] / (df["bedroomcnt"] + 1)

# Selected features
features = [
    "calculatedfinishedsquarefeet",
    "bedroomcnt",
    "bathroomcnt",
    "yearbuilt",
    "lotsizesquarefeet",
    "value_per_sqft",
    "bath_bed_ratio"
]

X = df[features]
y = np.log1p(df[target])

# Train-test split for hyperparameter tuning
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define pipelines
pipelines = {
    "RandomForest": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", RandomForestRegressor(random_state=42)),
    ]),
    "GradientBoosting": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", GradientBoostingRegressor(random_state=42)),
    ]),
    "XGBoost": Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("model", XGBRegressor(random_state=42, objective="reg:squarederror")),
    ]),
}

# Hyperparameter grids
param_grids = {
    "RandomForest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [10, 20, None],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2],
    },
    "GradientBoosting": {
        "model__n_estimators": [100, 200],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 5],
        "model__subsample": [0.8, 1.0],
    },
    "XGBoost": {
        "model__n_estimators": [100, 200],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 5],
        "model__subsample": [0.8, 1.0],
        "model__colsample_bytree": [0.8, 1.0],
    },
}

# Hyperparameter tuning
def tune_model(name, pipeline, param_grid, X_train, y_train):
    print(f"\nTuning {name}...")
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_grid,
        n_iter=10,
        scoring="neg_mean_absolute_error",
        cv=3,
        verbose=0,
        random_state=42,
        n_jobs=1
    )
    search.fit(X_train, y_train)
    return search.best_estimator_, search.best_params_

# Evaluate with Repeated CV
def evaluate_model(model, X, y):
    cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
    mae_scores = cross_val_score(
        model, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=1
    )
    mae_scores = -mae_scores
    return mae_scores.mean(), mae_scores.std()

# Run all models and collect results
results = {}
for name in pipelines:
    best_model, best_params = tune_model(name, pipelines[name], param_grids[name], X_train, y_train)
    mean_mae, std_mae = evaluate_model(best_model, X, y)
    results[name] = {
        "best_params": best_params,
        "mean_MAE": mean_mae,
        "std_MAE": std_mae
    }

# Print formatted report
print("\n=== Final CV Results (MAE) ===")
for model, vals in results.items():
    print(f"\nModel: {model}")
    print(" Best Hyperparameters:", vals["best_params"])
    print(f" Repeated CV MAE: {vals['mean_MAE']:.4f} ± {vals['std_MAE']:.4f}")

### Part 4: Discussion [3 pts]

Reflect on your tuning process and final results:

- What was your tuning strategy for each model? Why did you choose those hyperparameters?
- Did you find that certain types of preprocessing or feature engineering worked better with specific models?


After completing the tuning process and evaluating each model with repeated cross‑validation, we found that the boosting methods, particularly XGBoost, responded best to both hyperparameter optimization and our engineered features. Random Forest was more stable and required less tuning effort, but it did not achieve the same level of accuracy. Gradient Boosting improved with careful adjustments to the learning rate and number of estimators, but XGBoost’s additional flexibility with feature and sample subsampling gave it a slight edge. Based on these results, we selected XGBoost as our final model, as it consistently achieved the lowest MAE and demonstrated strong generalization across folds.

### Part 5: Final Model and Design Reassessment [6 pts]

In this part, you will finalize your best-performing model.  You’ll also consolidate and present the key code used to run your model on the preprocessed dataset.
**Requirements:**

- Decide one your final model among the three contestants.

- Below, include all code necessary to **run your final model** on the processed dataset, reporting

    - Mean and standard deviation of CV MAE Score.
    
    - Test score on held-out test set.




In [ ]:
# Add as many cells as you need


### Part 5: Discussion [8 pts]

In this final step, your goal is to synthesize your entire modeling process and assess how your earlier decisions influenced the outcome. Please address the following:

1. Model Selection:
- Clearly state which model you selected as your final model and why.

- What metrics or observations led you to this decision?

- Were there trade-offs (e.g., interpretability vs. performance) that influenced your choice?

2. Revisiting an Early Decision

- Identify one specific preprocessing or feature engineering decision from Milestone 1 (e.g., how you handled missing values, how you scaled or encoded a variable, or whether you created interaction or polynomial terms).

- Explain the rationale for that decision at the time: What were you hoping it would achieve?

- Now that you've seen the full modeling pipeline and final results, reflect on whether this step helped or hindered performance. Did you keep it, modify it, or remove it?

- Justify your final decision with evidence—such as validation scores, visualizations, or model diagnostics.

3. Lessons Learned

- What insights did you gain about your dataset or your modeling process through this end-to-end workflow?

- If you had more time or data, what would you explore next?

> Your text here